In [1]:
import xarray as xr
import os
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

Define a function to get a list of all the files in a directory

In [2]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    # https://thispointer.com/python-how-to-get-list-of-files-in-directory-and-sub-directories/
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

Specify the directory our geotiff files are stored within:

In [3]:
directory = r'C:\Users\steve\Desktop\snowmap_test8-20200423T000031Z-001\snowmap_test8'

Get a list of all the files in this directory:

In [4]:
file_list = getListOfFiles(directory)

Specify the filename we want to give the merged data:

In [5]:
out_filename = 'Example_PM_SSUM_25km_month02_.nc'

Now open each file, unpack the 5 "bands" of data, merge, and save to a new NetCDF file. 

(this is probably not the most efficient way to do this, but it seems to work)

In [6]:
lst = [];
tcc = [];
swe = [];
ndsi = [];
sensorclass = [];

for file in file_list:
    print("\rOpening {}".format(str(file)),end='')
    data = xr.open_rasterio(file)
    lst.append( xr.DataArray(data[0], name='lst') )
    tcc.append( xr.DataArray(data[1], name='tcc') )
    swe.append( xr.DataArray(data[2], name='swe') )
    ndsi.append( xr.DataArray(data[3], name='ndsi') )
    sensorclass.append( xr.DataArray(data[4], name='sensorclass') )

ds_lst = xr.merge(lst).drop('band')
ds_tcc = xr.merge(tcc).drop('band')
ds_swe = xr.merge(swe).drop('band')
ds_ndsi = xr.merge(ndsi).drop('band')
ds_sensorclass = xr.merge(sensorclass).drop('band')

ds = xr.merge([ds_lst, ds_tcc, ds_swe, ds_ndsi, ds_sensorclass])

ds.to_netcdf(out_filename)

Opening C:\Users\steve\Desktop\snowmap_test8-20200423T000031Z-001\snowmap_test8\snowmap_90_90.tifff

---
